# A Demo of Extreme Value Distributions

## Lib imports

In [ ]:
import numpy as np

import plotly
import plotly.plotly as pty
from plotly.tools import FigureFactory as ff

# Work in offline-mode for notebooks
plotly.offline.init_notebook_mode()

# plotly version
plotly.__version__

## Example distribution plot

In [ ]:
x = np.random.randn(10000)  
hist_data = [x]
group_labels = ['distplot']

fig = ff.create_distplot(hist_data, group_labels)
plotly.offline.iplot(fig, validate=False)